## 1) Preliminary analysis/preparation of aggregation

In [309]:
!pip3 install pandas
import pandas as pd
import os

raw_data_dir = os.path.join('..', 'data', 'raw')
inspection = os.path.join(raw_data_dir, 'inspection.csv')
inspection_order = os.path.join(raw_data_dir, 'order.csv')

inspection_df = pd.read_csv(inspection)
inspection_df = inspection_df.rename(columns={
    'originatingservicerequestnumber': 'Originating_Service_Request_Number',
    'InspectionCustomer': 'Inspection_Customer',
    'ElevatingDevicesNumber': 'Elevating_Devices_Number',
    'InspectionNumber': 'Inspection_Number',
    'InspectionLocation': 'Inspection_Location',
    'InspectionType': 'Inspection_Type',
    'Earliest_INSPECTION_Date': 'Earliest_Inspection_Date',
    'Latest_INSPECTION_Date': 'Latest_Inspection_Date',
    'InspectionOutcome': 'Inspection_Outcome'
})

inspection_order_df = pd.read_csv(inspection_order)
inspection_order_df = inspection_order_df.rename(columns={
    'ElevatingDevicesNumber': 'Elevating_Devices_Number',
    'TSSAStandardOrderNumber': 'TSSA_Standard_Order_Number',
    'RegulationReference': 'Regulation_Reference',
    'ClauseNumber': 'Clause_Number',
    'ClauseText': 'Clause_Text',
    'DIRECTIVE': 'Directive',
    'Inspectionsadditionalinformation': 'Inspections_Additional_Information',
    'RISKSCORE': 'Risk_Score',
    'Inspection_type': 'Inspection_Type',
    'DateofIssue': 'Date_of_Issue',
    'StatusofInspectionOrder': 'Status_of_Inspection_Order',
    'inspectionnumber': 'Inspection_Number',
    'DaystoComply': 'Days_to_Comply',
    'ComplianceDate': 'Compliance_Date',
    'customerorderedtocomply': 'Customer_Ordered_to_Comply'
})

print(inspection_order_df.columns)


Index(['Elevating_Devices_Number', 'TSSA_Standard_Order_Number',
       'Regulation_Reference', 'Clause_Number', 'Clause_Text', 'Directive',
       'Inspections_Additional_Information', 'Risk_Score', 'Inspection_Type',
       'Date_of_Issue', 'Status_of_Inspection_Order', 'Inspection_Number',
       'Days_to_Comply', 'Compliance_Date', 'Customer_Ordered_to_Comply'],
      dtype='object')


### a) Inspection Dataset and Inspection Order Dataset Matching: 

In [310]:
# Check for missing inspections
missing_inspections = inspection_df[~inspection_df['Inspection_Number'].isin(inspection_order_df['Inspection_Number'])]
print(f"Total missing inspections: {len(missing_inspections)}")

# Verify inspections with associated orders
has_associated_orders = inspection_df['Inspection_Number'].isin(inspection_order_df['Inspection_Number'])
inspection_df['Has Associated Orders'] = has_associated_orders

# Verify orders with associated inspections
has_associated_inspections = inspection_order_df['Inspection_Number'].isin(inspection_df['Inspection_Number'])
inspection_order_df['Has_Associated_Inspections'] = has_associated_inspections

# Print summary
print(f"Inspections with associated orders: {has_associated_orders.sum()}")
print(f"Orders with associated inspections: {has_associated_inspections.sum()}")

Total missing inspections: 95570
Inspections with associated orders: 47611
Orders with associated inspections: 162172


### b) Sort Datasets by Time

In [311]:
sorted_inspections = inspection_df.sort_values(by=['Earliest_Inspection_Date'])

sorted_inspections_order = inspection_order_df.sort_values(by=['Date_of_Issue'])

print(sorted_inspections_order.head(20))
print(sorted_inspections.tail(20))

        Elevating_Devices_Number TSSA_Standard_Order_Number  \
35384                       2110                        NaN   
89927                      37591                        NaN   
43513                      22202                        NaN   
89928                      37591                        NaN   
43517                      22202                        NaN   
43514                      22202                        NaN   
89313                      37592                        NaN   
43439                      22203                        NaN   
125368                     68476                        NaN   
126644                     68248                        NaN   
126645                     68248                        NaN   
126643                     68248                        NaN   
125546                     68477                        NaN   
125545                     68477                        NaN   
80142                      35068                       

### c) Inspection Orders Dataset:

In [312]:
sorted_inspections_order['Directive'] = sorted_inspections_order['Directive'].astype(str).where(sorted_inspections_order['Directive'].notnull(), 'No Directive.')
sorted_inspections_order['Inspections_Additional_Information'] = sorted_inspections_order['Inspections_Additional_Information'].astype(str).where(sorted_inspections_order['Inspections_Additional_Information'].notnull(), 'No Additional Information.')

sorted_inspections_order['Combined_Inspection_Info'] = (
    sorted_inspections_order['Directive'].str.replace(r'[\*\_\-]', ' ', regex=True).str.strip() + ' ' +
    sorted_inspections_order['Inspections_Additional_Information'].str.replace(r'[\*\_\-]', ' ', regex=True).str.strip()
)
print(sorted_inspections_order['Combined_Inspection_Info'].head(20))

35384     No Directive. Monthly Maintenance other:  All ...
89927     No Directive. submitt for alteration of device...
43513     No Directive. Monthly Maintenance other:  MAKE...
89928     No Directive. the fire recall keys shall be ke...
43517     No Directive. Monthly Maintenance other:  CLEA...
43514     No Directive. The annual test of the safeties ...
89313     No Directive. submitt for alteration of device...
43439     No Directive. THE OWNER SHALL REPORT TO TSSA W...
125368    No Directive. Make both in car emergency light...
126644    No Directive. oil log book other:  CLEAN THE PIT.
126645    No Directive. Car enclosure other:  RETURN THE...
126643    No Directive. Car enclosure other:  RE FASTEN/...
125546    No Directive. Car enclosure other:  REPLACE TH...
125545    No Directive. The car false ceiling shall be r...
80142     No Directive. Pit other:  remove non elevator ...
80141     No Directive. General  other  the alteration c...
80140     No Directive. General  other  

### d) Handling Missing "RISK SCORE" Values: 

The values are huge floats, so values have to be normalized to get a better idea and improve the model performance when used. Normalization will help to do meaningful conparisons and avoid bias. In this case the column is using very different scales and also NaN values. They will be converted to 

In [313]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

scaler = MinMaxScaler()

sorted_inspections_order['Risk_Score'] = pd.to_numeric(sorted_inspections_order['Risk_Score'], errors='coerce')
sorted_inspections_order['Risk_Score'] = sorted_inspections_order['Risk_Score'].where(sorted_inspections_order['Risk_Score'].notnull(), 0)

sorted_inspections_order['Risk_Score'] = scaler.fit_transform(sorted_inspections_order[['Risk_Score']])

test = sorted_inspections_order['Risk_Score'].unique()
print(test)

[7.38313896e-04 1.08286038e-03 1.13208131e-03 4.92209264e-05
 1.77195335e-03 8.36755748e-04 1.47662779e-04 7.87534822e-04
 1.03363945e-03 1.37818594e-03 8.85976675e-04 1.47662779e-03
 1.42740686e-03 1.67351150e-03 1.18130223e-03 0.00000000e+00
 4.48191227e-05 8.80779011e-04 2.24095613e-05 3.78143646e-06
 8.10066106e-05 2.53100437e-06 3.63804197e-06 1.62422412e-07
 5.30290510e-08 9.58488944e-09 1.81625218e-09 1.00902899e-10
 1.27482199e-09 3.13733859e-05 3.22782320e-07 5.23887566e-05
 1.49397076e-05 1.44217314e-10 1.25733016e-06 1.05541945e-04
 9.35807941e-09 1.79618594e-06 7.95435765e-08 6.22824828e-06
 3.25842533e-09 3.11338896e-04 1.30435455e-09 2.09743676e-04
 3.52518306e-07 6.10402834e-07 4.79006315e-04 7.88391266e-07
 6.81362281e-07 1.52421301e-06 5.56595325e-06 5.89621710e-07
 3.52934070e-05 1.39313792e-01 1.15136252e-02 1.86055102e-09
 2.66466325e-06 1.16245652e-07 8.20626267e-06 1.63820205e-05
 2.81830283e-06 4.30108018e-06 4.06352055e-06 7.62849149e-06
 5.20766911e-06 4.144402

### e) Inspection Outcome Variable (inspection dataset):

In [314]:
counts = sorted_inspections['Inspection_Outcome'].value_counts()

to_replace = counts[counts < 500].index

sorted_inspections['Inspection_Outcome'] = sorted_inspections['Inspection_Outcome'].replace(to_replace, 'Other')

grouped_counts = sorted_inspections['Inspection_Outcome'].value_counts()

print(grouped_counts)


Inspection_Outcome
Follow up              54605
Passed                 26064
DC Follow up           22302
All Orders Resolved    19555
Complete                7506
Shutdown                6110
Other                   2201
Follow up Major         1117
Follow up Sub Major     1002
Follow Up Initial        877
Unable to Inspect        689
Fail Initial             602
Passed Major             551
Name: count, dtype: int64


### f) Inspection Type Variable (inspection dataset): 

In [315]:
unique_values = sorted_inspections['Inspection_Type'].unique()
print(unique_values)
print('-------------')

corrections = {
    'ED-Sub  Inspection': 'Elevating Device Sub Inspection',
    'ED-FU Enforcement Action Insp': 'Elevating Device Followup Enforcement Action Inspection',
    'ED-Followup Lic Insp': 'Elevating Device -Followup License Inspection',
    'ED-Followup No-Lic Insp': 'Elevating Device Followup No-License Inspection',
    'ED-PWGSC Insp': 'Elevating Device PWGSC Inspection',
    'ED-PWGSC Foll-Up': 'Elevating Device PWGSC Follow-Up',
    'ED-Followup Ownership Change': 'Elevating Device Followup Ownership Change Inspection',
    'ED-Followup Minor Alt': 'ED-Followup Minor Alteration Inspection',
    'ED-MCP Follow up': 'Elevating Device MCP Follow-Up',
    'ED-MCP Enforcement Insp': 'Elevating Device MCP Enforcement Inspection',
    'ED-Perform L1 Incident Insp': 'Elevating Device Perform Level 1 Incident Inspection',
    'ED-Perform L1 Near Miss Insp': 'Elevating Device Perform Level 1 Near Miss Inspection',
    'ED-Inspection Temp Lic': 'Elevating Device Inspection Temporary License',
    'ED-Reg Non-Compliance': 'Elevating Device Regulatory Non-Compliance',
    'ED-Followup Reg Non-Compliance': 'Elevating Device Followup Regulatory Non-Compliance',
    'ED-Non-Mandated Insp ON': 'Elevating Device Non-Mandated Inspection Ontario',
    'ED-Non-Mandated Followup ON': 'Elevating Device Non-Mandated Follow-Up Ontario',
}

sorted_inspections['Inspection_Type'] = sorted_inspections['Inspection_Type'].replace(corrections)

print(sorted_inspections['Inspection_Type'].unique())

['ED-Followup Inspection' 'ED-Periodic Inspection' 'ED-Minor B Inspection'
 'ED-Sub Inspection' 'ED-Initial Inspection' 'ED-Enforcement Action'
 'ED-Minor A Inspection' 'ED-Sub  Inspection' 'ED-Unscheduled Inspection'
 'ED-Sub Inspection Major' 'ED-Sub Failed Initial'
 'ED-Major Alteration Inspection' 'ED-Followup Minor Alt'
 'ED-FU Enforcement Action Insp' 'ED-Followup Ownership Change'
 'ED-Followup Lic Insp' 'ED-MCP Follow up' 'ED-Re-Activate Inspection'
 'ED-Non-Mandated Insp ON' 'ED-MCP Enforcement Insp'
 'ED-Followup No-Lic Insp' 'ED-PWGSC Insp'
 'ED-Followup Reg Non-Compliance' 'ED-Inspection Temp Lic'
 'ED-Reg Non-Compliance' 'ED-PWGSC Foll-Up' 'ED-Non-Mandated Followup ON'
 'ED-Perform L1 Incident Insp' 'ED-Perform L1 Near Miss Insp']
-------------
['ED-Followup Inspection' 'ED-Periodic Inspection' 'ED-Minor B Inspection'
 'ED-Sub Inspection' 'ED-Initial Inspection' 'ED-Enforcement Action'
 'ED-Minor A Inspection' 'Elevating Device Sub Inspection'
 'ED-Unscheduled Inspection' 

### g) Create Dummy Variables (inspection dataset): 

In [316]:
dummies = pd.get_dummies(sorted_inspections[['Inspection_Outcome', 'Inspection_Type']], prefix='Dummy')
dummies = dummies.astype(int)  # Ensure the values are integers (0 and 1)
#print(dummies.head(20))

#Check worked well 
print(dummies['Dummy_All Orders Resolved'].value_counts())

Dummy_All Orders Resolved
0    123626
1     19555
Name: count, dtype: int64


## 2- Aggregation of the Inspection Dataset

### a & b) Group Data by Key Identifiers & Group Data by Key Identifiers:

In [317]:
sorted_inspections['Earliest_Inspection_Date'] = pd.to_datetime(sorted_inspections['Earliest_Inspection_Date'], errors='coerce')
sorted_inspections['Latest_Inspection_Date'] = pd.to_datetime(sorted_inspections['Latest_Inspection_Date'], errors='coerce')

grouped_inspections = sorted_inspections.groupby('Elevating_Devices_Number').agg({
    'Earliest_Inspection_Date': 'min',
    'Latest_Inspection_Date': 'max',
    'Inspection_Number': lambda x: list(x),
    'Inspection_Customer': 'first'
}).reset_index()

print(grouped_inspections.head(5))

   Elevating_Devices_Number Earliest_Inspection_Date Latest_Inspection_Date  \
0                         8               2012-03-05             2015-03-27   
1                         9               2012-03-05             2015-03-27   
2                        10               2012-03-05             2015-03-27   
3                        11               2012-03-05             2015-03-27   
4                        13               2012-03-06             2015-04-30   

                                   Inspection_Number  \
0  [3930603, 4076073, 3747295, 5248300, 3132984, ...   
1  [3930608, 4076078, 3747272, 5248304, 2708468, ...   
2  [4184747, 5248292, 5312245, 3930613, 4076085, ...   
3  [4184734, 3930616, 4076090, 3747339, 5248299, ...   
4  [3930620, 4076117, 3749130, 3763512, 5248307, ...   

                                 Inspection_Customer  
0  LEGISLATIVE ASSEMBLY OF ONTARIO ATTN:  JOHN ED...  
1  LEGISLATIVE ASSEMBLY OF ONTARIO ATTN:  JOHN ED...  
2  LEGISLATIVE ASSEMBLY

### c) Format Data for Current Inspection Outcome: Separate the current inspection

In [318]:
grouped_outcomes = sorted_inspections.groupby(['Elevating_Devices_Number', 'Inspection_Outcome']) \
    .size().reset_index(name='Count')

pivoted_outcomes = grouped_outcomes.pivot(
    index='Elevating_Devices_Number',
    columns='Inspection_Outcome',
    values='Count'
).fillna(0).astype(int).reset_index()

final_df = pd.merge(grouped_inspections, pivoted_outcomes, on='Elevating_Devices_Number', how='left')
final_df.columns = [col.replace(" ", "_") for col in final_df.columns]

print(final_df.head(1))

   Elevating_Devices_Number Earliest_Inspection_Date Latest_Inspection_Date  \
0                         8               2012-03-05             2015-03-27   

                                   Inspection_Number  \
0  [3930603, 4076073, 3747295, 5248300, 3132984, ...   

                                 Inspection_Customer  All_Orders_Resolved  \
0  LEGISLATIVE ASSEMBLY OF ONTARIO ATTN:  JOHN ED...                    0   

   Complete  DC_Follow_up  Fail_Initial  Follow_Up_Initial  Follow_up  \
0         0             0             0                  0          5   

   Follow_up_Major  Follow_up_Sub_Major  Other  Passed  Passed_Major  \
0                0                    0      0       1             0   

   Shutdown  Unable_to_Inspect  
0         0                  0  


### d) Filter Data Based on Inspection Order Dataset: A

In [321]:
inspection_numbers_set = set(sorted_inspections_order['Inspection_Number'])

filtered_inspections = final_df[final_df['Inspection_Number'].apply(lambda x: any(num in inspection_numbers_set for num in x))]

print(filtered_inspections.head(5))
print(f"Total rows after filter: {len(filtered_inspections)}")


object
int64
Total inspections: 0
